In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
@author: pho
"""
import sys
from threading import Thread
# Setting the Qt bindings for QtPy
import os
os.environ["QT_API"] = "pyqt5"
from pathlib import Path

# from PyQt5 import QtWidgets
# from PyQt5 import QtWidgets

import PyQt5
import PyQt5.QtWidgets
from PyQt5.QtWidgets import (QWidget, QSlider, QHBoxLayout,
                             QLabel, QApplication, QAction, QMainWindow, QFrame, QVBoxLayout)
from PyQt5.QtCore import Qt


# GUI Stuff
import pyvista as pv
from pyvistaqt import QtInteractor, MainWindow
import pyvistaqt as pvqt


In [ ]:
# from qtrangeslider import QRangeSlider

# as usual:
# you must create a QApplication before create a widget.
# range_slider = QRangeSlider()


In [ ]:
class MyMainWindow(MainWindow):

    def __init__(self, parent=None, show=True):
        QMainWindow.__init__(self, parent)

        # create the frame
        self.frame = QFrame()
        vlayout = QVBoxLayout()

        # add the pyvista interactor object
        self.plotter = QtInteractor(self.frame)
        vlayout.addWidget(self.plotter.interactor)
        self.signal_close.connect(self.plotter.close)

        self.frame.setLayout(vlayout)
        self.setCentralWidget(self.frame)

        # simple menu to demo functions
        mainMenu = self.menuBar()
        fileMenu = mainMenu.addMenu('File')
        exitButton = QAction('Exit', self)
        exitButton.setShortcut('Ctrl+Q')
        exitButton.triggered.connect(self.close)
        fileMenu.addAction(exitButton)

        # allow adding a sphere
        meshMenu = mainMenu.addMenu('Mesh')
        self.add_sphere_action = QAction('Add Sphere', self)
        self.add_sphere_action.triggered.connect(self.add_sphere)
        meshMenu.addAction(self.add_sphere_action)

        if show:
            self.show()

    def add_sphere(self):
        """ add a sphere to the pyqt frame """
        sphere = pv.Sphere()
        self.plotter.add_mesh(sphere, show_edges=True)
        self.plotter.reset_camera()



In [ ]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MyMainWindow()
    sys.exit(app.exec_())

In [ ]:
mp = pvqt.MultiPlotter(nrows=2, ncols=2)
mp[0, 0].add_mesh(pv.Sphere())
mp[0, 1].add_mesh(pv.Cylinder())
mp[1, 0].add_mesh(pv.Cube())
mp[1, 1].add_mesh(pv.Cone())



In [ ]:
## Slider with Callback Function Example:

p = pvqt.BackgroundPlotter() # Use just like you would a pv.Plotter() instance

def on_slider_update_mesh(value):
    res = int(value)
    sphere = pv.Sphere(phi_resolution=res, theta_resolution=res)
    p.add_mesh(sphere, name='sphere', show_edges=True)
    return

p.add_slider_widget(on_slider_update_mesh, [5, 100], title='Resolution')
p.show()


In [ ]:
# mesh points
vertices = np.array([[0, 0, 0],
                     [1, 0, 0],
                     [1, 1, 0],
                     [0, 1, 0],
                     [0.5, 0.5, -1]])

# mesh faces
faces = np.hstack([[4, 0, 1, 2, 3],  # square
                   [3, 0, 1, 4],     # triangle
                   [3, 1, 2, 4]])    # triangle

surf = pv.PolyData(vertices, faces)

# plot each face with a different color
# surf.plot(scalars=np.arange(3), cpos=[-1, 1, 0.5])
surf.plot(scalars=np.arange(3), cpos=[-1, 1])


In [ ]:
## Cylindrical Plotting Test:
## From https://notebook.community/nicoguaro/AdvancedMath/notebooks/vector_calculus-pyvista
red = (0.9, 0.1, 0.1)
blue = (0.2, 0.5, 0.7)
green = (0.3, 0.7, 0.3)

# Cylinder
phi, z = np.mgrid[0:2*np.pi:31j, -2:2*np.pi:31j]
x = 2*np.cos(phi)
y = 2*np.sin(phi)
cylinder = pv.StructuredGrid(x, y, z)

# Vertical plane
rho, z = np.mgrid[0:3:31j, -2:2*np.pi:31j]
x = rho*np.cos(np.pi/4)
y = rho*np.sin(np.pi/4)
vert_plane = pv.StructuredGrid(x, y, z)

# Horizontal plane
rho, phi = np.mgrid[0:3:31j, 0:2*np.pi:31j]
x = rho*np.cos(phi)
y = rho*np.sin(phi)
z = np.ones_like(x)
hor_plane = pv.StructuredGrid(x, y, z)

# view(geometries=[cylinder, vert_plane, hor_plane],
#      geometry_colors=[blue, red, green])

# scene = Scene([cylinder, vert_plane, hor_plane])

# cylinder.to_surface()

# cylinder.plot()

plotter = pv.Plotter(notebook=False) # Use just like you would a pv.Plotter() instance
plotter.add_mesh(cylinder, color=blue)
plotter.add_mesh(vert_plane, color=red)
plotter.add_mesh(hor_plane, color=green)


plotter.show()

In [ ]:
cyl_geo = pv.Cylinder()
# Compute the normals in-place and use them to warp the globe
normals = cyl_geo.compute_normals(inplace=True)  # this activates the normals as well

centers = cyl_geo.cell_centers()
# Now use those normals to warp the surface
# warp = mesh.warp_by_scalar(factor=0.5e-5)


p = pv.Plotter(notebook=True) # Use just like you would a pv.Plotter() instance
p = pvqt.BackgroundPlotter()
p.add_mesh(cyl_geo, color="tan", show_edges=True, render_points_as_spheres=True)
p.add_mesh(centers, color="red", point_size=8.0, render_points_as_spheres=True)
p.add_mesh(cyl_geo.points, color="green", point_size=5.0, render_points_as_spheres=True)
# p.show(jupyter_backend='panel')
# p.show(jupyter_backend='pythreejs')
# p.show(jupyter_backend=None)
p.show()